In [2]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50
import numpy as np


In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
optimizer=Adam(learning_rate=0.001)

In [5]:
train_datagen = ImageDataGenerator(validation_split=0.25,
        rescale=1./255,
        horizontal_flip=True,
        #width_shift_range = 0.2,
        #height_shift_range = 0.2
                                  )

validation_datagen = ImageDataGenerator(validation_split=0.25,
        rescale=1./255,
        horizontal_flip=True,
        #width_shift_range = 0.2,
        #height_shift_range = 0.2
                                       )

train_datagen_flow = train_datagen.flow_from_directory(
    'Faces/',
    target_size=(480, 640),
    batch_size=16,
    class_mode='sparse',
    subset='training',
    seed=12345)

val_datagen_flow = validation_datagen.flow_from_directory(
    'Faces/',
    target_size=(480, 640),
    batch_size=16,
    class_mode='sparse',
    subset='validation',
    seed=12345) 

Found 189 images belonging to 3 classes.
Found 62 images belonging to 3 classes.


In [6]:
features, target = next(train_datagen_flow)

In [7]:
target

array([2., 0., 1., 2., 2., 2., 2., 1., 1., 0., 0., 0., 1., 0., 0., 1.],
      dtype=float32)

In [8]:
from matplotlib import pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.add_subplot(4,4, 1)
print(target[0])
plt.imshow(features[0])

In [11]:
backbone = ResNet50(input_shape=(480, 640, 3),
                   weights='imagenet', 
                   include_top=False)
model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(3, activation='softmax')) 
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [12]:
steps_per_epoch = len(train_datagen_flow)
validation_steps = len(val_datagen_flow)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 15, 20, 2048)      23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 3)                 6147      
                                                                 
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
model.fit(train_datagen_flow, 
              validation_data=val_datagen_flow,
              epochs=20,
              batch_size=1,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

In [191]:
import cv2

In [192]:
camera = cv2.VideoCapture(0)
return_value, image = camera.read()
cv2.imwrite('opencv.png', image)
del(camera)

In [193]:
import tensorflow as tf
image = tf.keras.preprocessing.image.load_img('opencv.png', target_size=(480,640))

In [194]:
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])

In [195]:
input_arr = input_arr.astype('float32') / 255

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.add_subplot(4,4, 1)
plt.imshow(input_arr[0])

In [197]:
prediction = model.predict(input_arr)
predicted_class = np.argmax(prediction, axis=-1)
predicted_class

1/1 [==============================] - 1s 614ms/step


array([2], dtype=int64)

In [198]:
prediction

array([[0.11142045, 0.07434054, 0.814239  ]], dtype=float32)

In [199]:
model.save('trained_keras.h5')